# Lab 5: Google Speech Commands

In [1]:
import tensorflow as tf
print(tf.sysconfig.get_build_info())
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

OrderedDict([('cpu_compiler', '/dt9/usr/bin/gcc'), ('cuda_compute_capabilities', ['sm_35', 'sm_50', 'sm_60', 'sm_70', 'sm_75', 'compute_80']), ('cuda_version', '11.8'), ('cudnn_version', '8'), ('is_cuda_build', True), ('is_rocm_build', False), ('is_tensorrt_build', True)])
Num GPUs Available:  0


## Imports

In [2]:
import copy
import wave
from pathlib import Path
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, Conv1D, AvgPool1D, MaxPool1D, ZeroPadding1D, BatchNormalization, Flatten, Dense, Activation
from keras.utils.data_utils import get_file
from keras.utils.np_utils import to_categorical

## Download, cache and extract Google Speech Commands

In [3]:
dataset_dir = Path('datasets')
if not (dataset_dir/'testing_list.txt').exists(): # Assume dataset already downloaded/extracted if testing list is present
    get_file(None, "http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz",
                    extract=True,
                    file_hash="6b74f3901214cb2c2934e98196829835",
                    cache_dir='.',
                    cache_subdir=dataset_dir)

2428923189/2428923189 [==============================] - 12s 0us/step


## Load raw spoken digits data from Google Speech Commands

In [12]:
# Classes to handle, ordered by label
CLASSES = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']

with (dataset_dir/'testing_list.txt').open() as f:
    testing_list = f.read().splitlines()

x_train = []
y_train = []
x_test = []
y_test = []

for recording in dataset_dir.glob(f'**/*.wav'):
    if not recording.parent.name in CLASSES: # Ignore unused classes
        continue
    label = CLASSES.index(recording.parent.name) # Assign class number
    
    with wave.open(str(recording)) as f: # Read wave file
        data = np.frombuffer(f.readframes(f.getnframes()), dtype=np.int16).copy() # As 16-bit signed integer
        
    data = data.astype(np.float32) # Convert to 32-bit floating-point
    data.resize((16000, 1)) # Resize to 1s (16kHz) with zero-padding, 1 channel

    if str(recording.relative_to(dataset_dir)) in testing_list: # Assign to test set if file in test list
        x_test.append(data)
        y_test.append(label)
    else:
        x_train.append(data)
        y_train.append(label)

x_train = np.array(x_train)
y_train = to_categorical(np.array(y_train))
x_test = np.array(x_test)
y_test = to_categorical(np.array(y_test))

## Prepare for inference with fixed-point Q7.9 samples by scaling input data accordingly

In [13]:
FIXED_POINT = 9
x_train /= 2**FIXED_POINT
x_test  /= 2**FIXED_POINT

## Export small dataset (250 random vectors)

In [14]:
perms = np.random.permutation(len(y_test))[0:250]
x_test_250 = x_test[perms]
y_test_250 = y_test[perms]
np.savetxt('x_test_gsc_250.csv', x_test_250.reshape((x_test_250.shape[0], -1)), delimiter=',', fmt='%s')
np.savetxt('y_test_gsc_250.csv', y_test_250, delimiter=',', fmt='%s')

## Build model M5

In [15]:
model = Sequential()

model.add(Input(shape=(16000, 1)))

model.add(Conv1D(filters=32, kernel_size=80, strides=16, activation='relu'))
model.add(BatchNormalization(momentum=0.1))
model.add(MaxPool1D(pool_size=4))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(BatchNormalization(momentum=0.1))

model.add(MaxPool1D(pool_size=4))

model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(BatchNormalization(momentum=0.1))
model.add(MaxPool1D(pool_size=4))

model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(BatchNormalization(momentum=0.1))

model.add(MaxPool1D(pool_size=4))

model.add(Flatten())
model.add(Dense(units=len(CLASSES)))
model.add(Activation('softmax')) # SoftMax activation needs to be separate from Dense to remove it later on

# EXPLORE Learning Rate
opt = tf.keras.optimizers.Adam(learning_rate=10e-3)
model.summary()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_12 (Conv1D)          (None, 996, 32)           2592      
                                                                 
 batch_normalization_4 (Batc  (None, 996, 32)          128       
 hNormalization)                                                 
                                                                 
 max_pooling1d_12 (MaxPoolin  (None, 249, 32)          0         
 g1D)                                                            
                                                                 
 conv1d_13 (Conv1D)          (None, 247, 32)           3104      
                                                                 
 batch_normalization_5 (Batc  (None, 247, 32)          128       
 hNormalization)                                                 
                                                      

## Train model

In [16]:
model.fit(x_train, y_train, epochs=70, batch_size=384, validation_data=(x_test, y_test))

Epoch 1/70
91/91 [==============================] - 54s 555ms/step - loss: 1.1434 - categorical_accuracy: 0.6200 - val_loss: 0.7126 - val_categorical_accuracy: 0.7616
Epoch 2/70
91/91 [==============================] - 50s 550ms/step - loss: 0.4883 - categorical_accuracy: 0.8397 - val_loss: 0.4420 - val_categorical_accuracy: 0.8578
Epoch 3/70
91/91 [==============================] - 50s 553ms/step - loss: 0.3492 - categorical_accuracy: 0.8874 - val_loss: 0.3437 - val_categorical_accuracy: 0.8931
Epoch 4/70
91/91 [==============================] - 50s 553ms/step - loss: 0.2817 - categorical_accuracy: 0.9067 - val_loss: 0.3159 - val_categorical_accuracy: 0.9050
Epoch 5/70
91/91 [==============================] - 50s 552ms/step - loss: 0.2425 - categorical_accuracy: 0.9188 - val_loss: 0.3425 - val_categorical_accuracy: 0.8943
Epoch 6/70
12/91 [==>...........................] - ETA: 42s - loss: 0.2357 - categorical_accuracy: 0.9210

KeyboardInterrupt: ignored

## Evaluate model on test dataset

In [17]:
model.evaluate(x_test, y_test, verbose=2)
pred_test = model.predict(x_test)
print(tf.math.confusion_matrix(y_test.argmax(axis=1), pred_test.argmax(axis=1)))

129/129 - 1s - loss: 0.3127 - categorical_accuracy: 0.9097 - 1s/epoch - 10ms/step
129/129 [==============================] - 2s 11ms/step
tf.Tensor(
[[387   1  13   2   4   0   4   3   2   2]
 [  5 364   2   3   7   3   2   1   1  11]
 [  6   0 379  14   1   0   1   0  22   1]
 [  2   0  22 350   0   1   8   2  19   1]
 [  7   6  16   5 344   9   4   2   6   1]
 [  0   4   1   7   7 409   1   8   5   3]
 [  2   0   4   5   0   0 376   0   7   0]
 [  8   1   8   2   1   8   3 371   2   2]
 [  0   3   7   8   0   0   2   0 386   2]
 [  4  18   1   1   1  10   1   0   2 370]], shape=(10, 10), dtype=int32)


## Evaluate model on small dataset

In [18]:
model.evaluate(x_test_250, y_test_250, verbose=2)
pred_test_250 = model.predict(x_test_250)
print(tf.math.confusion_matrix(y_test_250.argmax(axis=1), pred_test_250.argmax(axis=1)))

8/8 - 0s - loss: 0.2137 - categorical_accuracy: 0.9360 - 121ms/epoch - 15ms/step
8/8 [==============================] - 0s 12ms/step
tf.Tensor(
[[22  0  1  0  1  0  0  0  0  0]
 [ 0 22  0  0  0  0  0  0  0  1]
 [ 0  0 25  1  0  0  0  0  2  0]
 [ 0  0  1 27  0  0  1  0  0  0]
 [ 0  0  1  0 16  0  0  0  0  0]
 [ 0  0  0  1  0 21  0  2  0  0]
 [ 0  0  0  0  0  0 23  0  0  0]
 [ 1  0  0  0  0  0  0 30  0  0]
 [ 0  0  1  0  0  0  0  0 23  0]
 [ 1  1  0  0  0  0  0  0  0 25]], shape=(10, 10), dtype=int32)


## Save trained model

In [ ]:
model.save('lab_gsc.h5')

## Remove SoftMax layer

In [ ]:
model = tf.keras.Model(model.input, model.layers[-2].output, name=model.name)

## Install MicroAI for C inference code generation (kerascnn2c module)

In [ ]:
%pip install https://bitbucket.org/edge-team-leat/microai_public/get/6adfbcb347d3.zip#subdirectory=third_party/kerascnn2c_fixed
import kerascnn2c

## Generate C code for the trained model with 16-bit fixed-point representation

In [ ]:
res = kerascnn2c.Converter(output_path=Path('gsc_output_fixed'),
                           fixed_point=FIXED_POINT, # Number of bits for the fractional part, Q7.9 format
                           number_type='int16_t', # Data type for weights/activations (16 bits quantization)
                           long_number_type='int32_t', # Data type for intermediate results
                           number_min=-(2**15), # Minimum value for 16-bit signed integers
                           number_max=(2**15)-1 # Maximum value for 16-bit signed integers
                          ).convert_model(copy.deepcopy(model))
with open('gsc_model_fixed.h', 'w') as f:
    f.write(res)

## Compile the 16-bit fixed-point C code for x86 and evaluate on small dataset

In [ ]:
!g++ -Wall -Wextra -pedantic -Ofast -o gsc_fixed -Igsc_output_fixed/ gsc_output_fixed/model.c main.cpp 
!./gsc_fixed x_test_gsc_250.csv y_test_gsc_250.csv